In [1]:
pip install tabula-py

     |████████████████████████████████| 12.0 MB 13.2 MB/s eta 0:00:01   |██████████▏                     | 3.8 MB 6.9 MB/s eta 0:00:02     |████████████████▊               | 6.3 MB 6.9 MB/s eta 0:00:01
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
import tabula

In [15]:
url = "https://machikanekun-ticket.jp/files/shop/shoplist.pdf"
dfs = tabula.read_pdf(url, stream=True , pages = '10')

In [16]:
dfs[0].head(5)

,全店(○)...全店舗共通券[紙] 中小(●)...中小店舗専用券[紙],デジ (★)...デジタル商品券[中小店舗専用券],Unnamed: 0,Unnamed: 1
0,利用可能な商品券,NaN,NaN,NaN
1,店舗名 業種,NaN,所在地,電話番号
2,全店,中小 デジ,NaN,NaN
3,236 クロロフイル豊中美顔教室 小売業 その他 〇,● ★,本町1-2-6 モニュマ豊中ビル4F,0120-06-4966
4,237 ワンペニー 飲食店 〇,● ★,本町1-3-19 長尾第一ビル1F E,070-3789-5350


In [17]:
dfs[0].tail(5)

,全店(○)...全店舗共通券[紙] 中小(●)...中小店舗専用券[紙],デジ (★)...デジタル商品券[中小店舗専用券],Unnamed: 0,Unnamed: 1
38,271 上野治療院 医療福祉 〇,● ★,本町3-1-3,06-6852-3537
39,272 お好み幸房 和楽 飲食店 〇,● ★,本町3-13-1,06-6844-0766
40,273 ステーキハウス18 飲食店 〇,● ★,本町3-13-3 B1F,06-4867-4444
41,274 北のしゃぶしゃぶ 飲食店 〇,● ★,本町3-13-3 B1F,06-6853-3232
42,275 ココカラファイン 豊中本町店 ドラッグストア 〇,NaN,本町3-13-6,06-4865-3511


1. ヘッダ行（最初の3行）を削除
2. 列目を整形：連番の取得、店名を取得、カテゴリと〇の有無、分割うまくいっているかチェック
3. 2列目を整形：●と☆を分割
4. 3列目を整形：住所を取得、建物名などは分割

### ヘッダ業を削除

In [23]:
df_org = dfs[0]
df = df_org.iloc[3:,:]
columns = df.columns
df.head(5)

,全店(○)...全店舗共通券[紙] 中小(●)...中小店舗専用券[紙],デジ (★)...デジタル商品券[中小店舗専用券],Unnamed: 0,Unnamed: 1
3,236 クロロフイル豊中美顔教室 小売業 その他 〇,● ★,本町1-2-6 モニュマ豊中ビル4F,0120-06-4966
4,237 ワンペニー 飲食店 〇,● ★,本町1-3-19 長尾第一ビル1F E,070-3789-5350
5,238 CO.ZARA 飲食店 〇,● ★,本町1-3-19 長尾第一ビル1F-B,080-3103-9802
6,239 酒と蕎麦こなから 飲食店 〇,● ★,本町1-3-19 長尾第一ビル1F-G,06-6868-9932
7,240 司法書士スタンダード 豊中登記相談センター その他の業種 〇,● ★,本町1-3-19 長尾第一ビル3F-A-2,06-6151-4422


In [56]:
print(df.iloc[1,0].split(' ', 1)[0]) # 通し番号
print(df.iloc[1,0].split(' ', 1)[1].rsplit(' ', 2)[0]) # 店舗名
print(df.iloc[1,0].split(' ', 1)[1].rsplit(' ', 2)[1]) # 業態
print('〇' in df.iloc[1,0]) # 〇 全店

237
ワンペニー
飲食店
True


In [53]:
print(df.iloc[1,2].split(' ', 1)[0]) # 住所１
print(df.iloc[1,2].split(' ', 1)[1]) # 住所２

本町1-3-19
長尾第一ビル1F E


In [55]:
print(df.iloc[1,3]) # 電話番号

070-3789-5350


In [76]:
df.iloc[:,1].head()

3    ● ★
4    ● ★
5    ● ★
6    ● ★
7    ● ★
Name: デジ (★)...デジタル商品券[中小店舗専用券], dtype: object

In [ ]:
def delete_header(df, header_rows=3):
    return(df.iloc[header_rows:,:])b

,全店(○)...全店舗共通券[紙] 中小(●)...中小店舗専用券[紙],デジ (★)...デジタル商品券[中小店舗専用券],Unnamed: 0,Unnamed: 1
3,236 クロロフイル豊中美顔教室 小売業 その他 〇,● ★,本町1-2-6 モニュマ豊中ビル4F,0120-06-4966
4,237 ワンペニー 飲食店 〇,● ★,本町1-3-19 長尾第一ビル1F E,070-3789-5350
5,238 CO.ZARA 飲食店 〇,● ★,本町1-3-19 長尾第一ビル1F-B,080-3103-9802
6,239 酒と蕎麦こなから 飲食店 〇,● ★,本町1-3-19 長尾第一ビル1F-G,06-6868-9932
7,240 司法書士スタンダード 豊中登記相談センター その他の業種 〇,● ★,本町1-3-19 長尾第一ビル3F-A-2,06-6151-4422


In [207]:
df['全店'] = df.iloc[:,0].str.contains('〇')
df['全店'] = df['全店'].replace(True, '〇')
df['デジタル'] = df.iloc[:,1].str.contains('★')
df['デジタル'] = df['デジタル'].replace(True, '★')
df['中小'] = df.iloc[:,1].str.contains('●')
df['中小'] = df['中小'].replace(True, '●')

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [209]:
df.tail()

,全店(○)...全店舗共通券[紙] 中小(●)...中小店舗専用券[紙],デジ (★)...デジタル商品券[中小店舗専用券],Unnamed: 0,Unnamed: 1,digital,chusho,all,shopname,category,Address1,Address2,Phone,全店,デジタル,中小
0,,,,,,,,,,,,,,,
271,271 上野治療院 医療福祉 〇,● ★,本町3-1-3,06-6852-3537,True,True,True,上野治療院,医療福祉,大阪府豊中市本町3-1-3,None,06-6852-3537,〇,★,●
272,272 お好み幸房 和楽 飲食店 〇,● ★,本町3-13-1,06-6844-0766,True,True,True,お好み幸房 和楽,飲食店,大阪府豊中市本町3-13-1,None,06-6844-0766,〇,★,●
273,273 ステーキハウス18 飲食店 〇,● ★,本町3-13-3 B1F,06-4867-4444,True,True,True,ステーキハウス18,飲食店,大阪府豊中市本町3-13-3,B1F,06-4867-4444,〇,★,●
274,274 北のしゃぶしゃぶ 飲食店 〇,● ★,本町3-13-3 B1F,06-6853-3232,True,True,True,北のしゃぶしゃぶ,飲食店,大阪府豊中市本町3-13-3,B1F,06-6853-3232,〇,★,●
275,275 ココカラファイン 豊中本町店 ドラッグストア 〇,NaN,本町3-13-6,06-4865-3511,NaN,NaN,True,ココカラファイン 豊中本町店,ドラッグストア,大阪府豊中市本町3-13-6,None,06-4865-3511,〇,NaN,NaN


In [92]:
df.index = df.iloc[:,0].str.split(' ', 1, expand=True)[0]

,全店(○)...全店舗共通券[紙] 中小(●)...中小店舗専用券[紙],デジ (★)...デジタル商品券[中小店舗専用券],Unnamed: 0,Unnamed: 1,digital,chusho,all
0,,,,,,,
236,236 クロロフイル豊中美顔教室 小売業 その他 〇,● ★,本町1-2-6 モニュマ豊中ビル4F,0120-06-4966,True,True,True
237,237 ワンペニー 飲食店 〇,● ★,本町1-3-19 長尾第一ビル1F E,070-3789-5350,True,True,True
238,238 CO.ZARA 飲食店 〇,● ★,本町1-3-19 長尾第一ビル1F-B,080-3103-9802,True,True,True
239,239 酒と蕎麦こなから 飲食店 〇,● ★,本町1-3-19 長尾第一ビル1F-G,06-6868-9932,True,True,True


In [248]:
def extract_shopname(df):
    shopnames = []
    categories = []
    tmp_series = df.iloc[:,0].str.split(' ', 1, expand=True)[1].str.replace('〇', '').str.rstrip(' ')
    print(tmp_series)
    for x in tmp_series:
        print(x)
        if x.endswith('その他'):
            shopname = x.rsplit(' ', 2)[0].rstrip(' ')
            category = x.replace(shopname, '').lstrip(' ')
            #print(shopname, ',', category, 'True')
        else:
            shopname = x.rsplit(' ', 1)[0].rstrip(' ')
            category = x.replace(shopname, '').lstrip(' ')
            #print(shopname, ',', category, 'False')
        shopnames.append(shopname)
        categories.append(category)
    df['shopname'] = pd.Series(shopnames, index=df.index)
    df['category'] = pd.Series(categories, index=df.index)
    return df

In [155]:
df = extract_shopname(df)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [165]:
df['Address1'] = '大阪府豊中市' + df.iloc[:,2].str.split(' ', 1, expand=True)[0]
df['Address2'] = df.iloc[:,2].str.split(' ', 1, expand=True)[1]
df['Phone'] = df.iloc[:,3]
df.head()

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,全店(○)...全店舗共通券[紙] 中小(●)...中小店舗専用券[紙],デジ (★)...デジタル商品券[中小店舗専用券],Unnamed: 0,Unnamed: 1,digital,chusho,all,shopname,category,Address1,Address2,Phone
0,,,,,,,,,,,,
236,236 クロロフイル豊中美顔教室 小売業 その他 〇,● ★,本町1-2-6 モニュマ豊中ビル4F,0120-06-4966,True,True,True,クロロフイル豊中美顔教室,小売業 その他,大阪府豊中市本町1-2-6,モニュマ豊中ビル4F,0120-06-4966
237,237 ワンペニー 飲食店 〇,● ★,本町1-3-19 長尾第一ビル1F E,070-3789-5350,True,True,True,ワンペニー,飲食店,大阪府豊中市本町1-3-19,長尾第一ビル1F E,070-3789-5350
238,238 CO.ZARA 飲食店 〇,● ★,本町1-3-19 長尾第一ビル1F-B,080-3103-9802,True,True,True,CO.ZARA,飲食店,大阪府豊中市本町1-3-19,長尾第一ビル1F-B,080-3103-9802
239,239 酒と蕎麦こなから 飲食店 〇,● ★,本町1-3-19 長尾第一ビル1F-G,06-6868-9932,True,True,True,酒と蕎麦こなから,飲食店,大阪府豊中市本町1-3-19,長尾第一ビル1F-G,06-6868-9932
240,240 司法書士スタンダード 豊中登記相談センター その他の業種 〇,● ★,本町1-3-19 長尾第一ビル3F-A-2,06-6151-4422,True,True,True,司法書士スタンダード 豊中登記相談センター,その他の業種,大阪府豊中市本町1-3-19,長尾第一ビル3F-A-2,06-6151-4422


In [223]:
df.iloc[-1,10:]

Address2            None
Phone       06-4865-3511
全店                     〇
デジタル                 NaN
中小                   NaN
Name: 275, dtype: object

In [ ]:
delete_header(df_org).head(5)

In [265]:
def process_df(df):
    df = delete_header(df)
    df = df[~df.iloc[:,2].isnull()] # 地域行削除

    df.index = df.iloc[:,0].str.split(' ', 1, expand=True)[0]
    df.index.name = '番号'

    df = extract_shopname(df)

    df['全店'] = df.iloc[:,0].str.contains('〇').fillna(False)
    df['全店'] = df['全店'].replace(True, '〇')
    df['全店'] = df['全店'].replace(False, '')

    df['デジタル'] = df.iloc[:,1].str.contains('★').fillna(False)
    df['デジタル'] = df['デジタル'].replace(True, '★')
    df['デジタル'] = df['デジタル'].replace(False, '')
    
    df['中小'] = df.iloc[:,1].str.contains('●').fillna(False)
    df['中小'] = df['中小'].replace(True, '●')
    df['中小'] = df['中小'].replace(False, '')
    
    df['住所１'] = '豊中市' + df.iloc[:,2].str.split(' ', 1, expand=True)[0]
    df['住所２'] = df.iloc[:,2].str.split(' ', 1, expand=True)[1].fillna('')
    df['電話番号'] = df.iloc[:,3]
    
    return df.iloc[:,4:]

In [242]:
df = process_df(df_org)
df

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,shopname,category,全店,デジタル,中小,住所１,住所２,電話番号
番号,,,,,,,,
236,クロロフイル豊中美顔教室,小売業 その他,〇,★,●,大阪府豊中市本町1-2-6,モニュマ豊中ビル4F,0120-06-4966
237,ワンペニー,飲食店,〇,★,●,大阪府豊中市本町1-3-19,長尾第一ビル1F E,070-3789-5350
238,CO.ZARA,飲食店,〇,★,●,大阪府豊中市本町1-3-19,長尾第一ビル1F-B,080-3103-9802
239,酒と蕎麦こなから,飲食店,〇,★,●,大阪府豊中市本町1-3-19,長尾第一ビル1F-G,06-6868-9932
240,司法書士スタンダード 豊中登記相談センター,その他の業種,〇,★,●,大阪府豊中市本町1-3-19,長尾第一ビル3F-A-2,06-6151-4422
241,若林鍼灸院,医療福祉,〇,★,●,大阪府豊中市本町1-3-2,,06-6845-1177
242,カリトロ 豊中店,飲食店,〇,★,●,大阪府豊中市本町1-4-1,,06-6846-7833
243,ほっかほっか亭 豊中駅前店,小売業 その他,〇,★,●,大阪府豊中市本町1-4-3,,06-6853-3536
244,餃子の勝 豊中本店,飲食店,〇,★,●,大阪府豊中市本町1-4-4,,06-7175-1164


In [ ]:
url = "https://machikanekun-ticket.jp/files/shop/shoplist.pdf"
dfs = tabula.read_pdf(url, stream=True , pages = '4-50')

In [ ]:
dfs_processed = []
for i, df_org in enumerate(dfs):
    print(i)
    dfs_processed.append(process_df(df_org))

In [ ]:
df_processed = pd.concat(dfs_processed)
df_processed.to_csv('./shoplist20211122.csv')